In [8]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float_kind':lambda x: "%.3g" % x})
from itertools import product, permutations
import math
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
from pprint import pprint

import coverSolver
from coverSolver import *

In [14]:
pieces=getPieces()
board=getBoard()

basis=np.array([[1,0,0.5],
                [0,1,0.5],
                [0,0,2.0**.5/2]])

#many Shape and ShapeSet methods (those making use of a Shape's state representation) require the universe u global variable defined
global u
u=Universe(board,basis)


## Rotation of shapes on a lattice.

This is the part that I understand the least about solving the puzzle, but I will write down what I know.

If a rotation $M$ is an isomorphism of the lattice, it is a valid rotation for a shape (FIXME: It feels like it's also necessary for $M$ to be a lattice isomorphism, but I haven't proved it yet).

### Which orientations are valid?
Our piece can 'point' from the origin along the axes in the $X-Y$ plane, and also along the 8 diagnonals in the directions of 
$$
\begin{bmatrix}
         \pm 1\\
         \pm 1\\
         \pm 1\\
\end{bmatrix}
$$
For each direction, the piece can also be rotated around it's pointing direction by $180^{\circ}$ and still remain on the lattice.
So we have 24 possible orientations.   It **looks** like that's it, but I have no proof.  Our lattice looks like a [Monoclinic crystal system with a Oblique rhombic prism
unit cell](https://en.wikipedia.org/wiki/Monoclinic_crystal_system), and there are some references to groups which may explain more.

It looks like all orientations can be generated by rotating the piece by $90^{\circ}$ 0,1,2 or 3 times about the diagnoals in the $X-Y$ plane, or by rotating by $180^{\circ}$ 0 or 1 times about the z-axis.  So,

1. Pick an axis of rotation from 
$$
\begin{bmatrix}
         0\\
         0\\
         1\\
\end{bmatrix}
,
\begin{bmatrix}
         \frac{\sqrt{2}}{2}\\
         \frac{\sqrt{2}}{2}\\
         0\\
\end{bmatrix}
,
\begin{bmatrix}
         \frac{\sqrt{2}}{2}\\
         -\frac{\sqrt{2}}{2}\\
         0\\
\end{bmatrix}
$$
2. rotate about it by a multiple of $90^{\circ}$ if it's a diagonal or $180^{\circ}$ if it's the z axis.
3. Repeat from step 1 with one of the remaining axes.

The above gives 192 rotations, so some of the 24 valid orientations will be created more than once.

Any advice on how to do this smarter from an abstract algebraist or a crystallographer or a solid-state chemist is welcome.


In [476]:
#diagonals in the X-Y plane, are the rotation axes, with 4-fold symmetry each
#each row is a rotation axis
rots=rotations(np.array(
    [[1,1,0],
     [1,-1,0],
     [0,1,0]
    ]), 
    np.array([4,4,2]).T)

orientSet=ShapeSet()
for p in pieces:
    orientSet.extend(p.orientations(rots))


### Translations
For every orientation, we need to make a list of possible translations that fit inside the board. For a shape to fit inside the board, it's bounding box needs to fit inside the board's bounding box.

In [477]:
# stateSet=ShapeSet()
# for p in orientSet[:1]:
#     stateSet.extend(p.containedTranslations(u.board))
d=p.containedTranslations(u.board)


[[], [], []]
set(['\x01\x02\x00', '\x01\x02\x01', '\x00\x00\x01', '\x00\x00\x00', '\x00\x02\x01', '\x00\x02\x00', '\x00\x00\x03', '\x00\x00\x02', '\x01\x00\x02', '\x01\x00\x03', '\x01\x00\x00', '\x01\x00\x01', '\x00\x01\x02', '\x02\x02\x01', '\x02\x02\x00', '\x02\x00\x01', '\x02\x00\x00', '\x02\x00\x02', '\x01\x04\x00', '\x03\x00\x00', '\x03\x00\x01', '\x04\x04\x00', '\x02\x04\x00', '\x03\x02\x00', '\x03\x01\x00', '\x03\x04\x00', '\x04\x00\x00', '\x04\x02\x00', '\x00\x02\x02', '\x00\x01\x00', '\x00\x01\x01', '\x01\x03\x01', '\x01\x03\x00', '\x01\x01\x01', '\x01\x01\x00', '\x01\x01\x03', '\x01\x01\x02', '\x00\x03\x00', '\x00\x03\x01', '\x02\x03\x00', '\x02\x03\x01', '\x02\x02\x02', '\x03\x02\x01', '\x02\x01\x02', '\x02\x01\x00', '\x02\x01\x01', '\x00\x00\x04', '\x03\x01\x01', '\x00\x04\x00', '\x03\x03\x01', '\x03\x03\x00', '\x04\x01\x00', '\x04\x03\x00', '\x00\x01\x03', '\x01\x02\x02'])


In [478]:
orientSet

[Shape(array([[0, 0, 0],
        [0, 0, 1],
        [0, 0, 2],
        [0, 0, 3],
        [1, 1, 0]], dtype=int8), 0), Shape(array([[0, 0, 0],
        [0, 1, 0],
        [0, 2, 0],
        [0, 3, 0],
        [1, 1, 0]], dtype=int8), 0), Shape(array([[0, 0, 0],
        [1, 0, 0],
        [2, 0, 0],
        [3, 0, 0],
        [1, 1, 0]], dtype=int8), 0), Shape(array([[0, 0, 3],
        [0, 0, 2],
        [0, 0, 1],
        [0, 0, 0],
        [1, 1, 1]], dtype=int8), 0), Shape(array([[0, 0, 3],
        [0, 1, 2],
        [0, 2, 1],
        [0, 3, 0],
        [1, 1, 1]], dtype=int8), 0), Shape(array([[0, 0, 3],
        [1, 0, 2],
        [2, 0, 1],
        [3, 0, 0],
        [1, 1, 1]], dtype=int8), 0), Shape(array([[0, 0, 3],
        [1, 1, 2],
        [2, 2, 1],
        [3, 3, 0],
        [1, 1, 1]], dtype=int8), 0), Shape(array([[0, 0, 3],
        [1, 1, 2],
        [2, 2, 1],
        [3, 3, 0],
        [1, 1, 3]], dtype=int8), 0), Shape(array([[0, 1, 0],
        [1, 1, 0],
        [2, 

In [462]:
s=ShapeSet([ pieces[0]])
# s.visualize(individually=True)